# Consejos para data wrangling con Python  


**Naomi Ceder, @naomiceder**

- **Presidenta Saliente de la Python Software Foundation**
- **La autora del Quick Python Book, 3rd ed**
- **Dick Blick Art Materials**

* **Exploring Python twitch stream** - 2020-05-9 2 PM CDT, via https://www.twitch.tv/nceder/
* https://naomiceder.tech, @naomiceder

**Este cuaderno** (pero no los archivos de datos) está disponible en Github - 

## Cuidado - Spanglish ahead ;-)

## Agenda


* Introducción
  * Herriamientas
  * Buscar ayuda

* Problemas de muestra - código y explicación 

  * Archivos de texto
  * Fixed Width
  * Binary
  * CSV
  * JSON
  * XML
  
* Consejos generales
   * Redimiento
   * Cómo manejar archivos muy largos
   * Depuración


### El enfoque de este taller

* trabajar con varios ejemplos simples
* usar las herriemientas las más básicas posible 
* compartir mi experiencia

No va a ser prescriptivo ni un "cookbook"


## Filosofía del taller

“Todas las familias felices se parecen unas a otras, pero cada familia infeliz lo es a su manera.” - Tolstoy

“¿Qué podría salir mal?” - Naomi, and many others

“Si algo puede salir mal, lo hará” - Murphy


**o**

Si todo va bien, puede usar las herramientas de alto nivel, pero cuando las cosas salen mal, debe trabajar en un nivel inferior. Y muchas veces las cosas salen mal.




## Herriamientas

* Python 3.8+
* la librería estándar - `collections, sys, os, str, re` módulos
* `pip`
  * `conda` si usas una distribución de Anaconda
* los entornos virturales
  * virtualenv o conda-env
* pandas


### Por qué no siempre usamos pandas?

* pandas muchas veces no está adecuada para el uso en producción o pipelines
* pandas no es optimizada para el rendimiento máximo 
* ve la filosofía del taller 

### La documentación en Python.org 

* Python Tutorial - https://docs.python.org/3/tutorial/index.html / https://python-docs-es.readthedocs.io/es/3.8/tutorial/index.html
* La ~~librería~~biblioteca estándar - https://docs.python.org/3/library/index.html - (aún no se traduce) https://python-docs-es.readthedocs.io/es/3.8/library/index.html


## Los consejos generales

* Empieza con lo simple
* Toma en cuenta el propósito
* Haz las comprobaciones de sanidad frecuentes
* Limpiar los datos es destruir información, hazlo con cuidado

## Las técnicas básicas

* Usa los métodos incorporados -  str.translate, lower(), etc
* Los conjuntos (sets) son los más rápidos con `in`
* Usa funciones - códico comun debe ponerse en funciones
* Los decorators - útil para funciones que apoyan a otras funciones
* Usa la bibioteca estándar
* Las expresiones regulares - más potentes, pero más lentas, deberían compilarse
* Usa otras bibliotecas - de PyPI, Pandas, NumPy, etc
* Usa las comprensiones de listas para filtrar

## Archivos de text


* problemas con newline
  * los fines de líneas pueden ser '\n', '\r', '\r\n'
  * `strip()`, `rstrip()`,  depués añade los fines de líneas
  * `newline` - None, '', '\n', '\r', '\r\n'
* errors de codificación
  * abre el archivo en modo binário
  * `errors` = ignore, replace, or otra opción  
* null carácter - "x\00"
* como los otros formatos usan archivos de tipo text, estos problemas pueden ocurrir también con ellos

### strings vs. bytes
* En Python 2 no hubo ningún diferencia
  * cadena/string - una serie de bytes o carácteres de una cadena
  * se necesita codificación para unicode 
* una **enorme** diferencia en Python 3
  * bytes - una serie de bytes, pero en verdad solamente bytes (0-255 integers)
  * cadena/string - Unicode 
  * hay que convertir una a la otra, bytes no son cadenas!

In [ ]:
b = [0, 9, 32, 48, 65, 66, 67]
#b = [0, 9, 32, 48, 65, 66, 67, 255]
b_string = bytes(b)
print(b_string.decode())
b_string.decode()

In [ ]:
# text file example

open("test", "wb").write(bytes("this is\nä\x80\ff\r\ntest\xc3\x28", encoding='utf8')+bytes([255]))


In [ ]:
text = open("test", "r", newline='', errors="strict").read()

text = open("test", "r", newline='', errors="replace").read()
print(text)
text

In [ ]:
text2 = open("test2", "r").read()
print(text2)
text2

In [ ]:
open("test2", "wb").write(b"this,is,a\ntest\x00,null,file")
import csv
for x in csv.reader(open("test2", "r", errors="replace")):
    print(x)

# Cleaning NULL (\x00) bytes from a data file
def clean_null(filename):
    with open(filename, 'rb') as fi:
        data = fi.read()
        fi.close()
    with open(f'{filename}.new', 'wb') as fo:
        fo.write(data.replace(b'\x00', b''))
        fo.close()

clean_null("test2")
for x in csv.reader(open("test2.new", "r")):
     print(x)

## Fixed Width Records

Problema - clima de Londres
* plain python
* biblioteca FixedWidth
* pandas

http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_fwf.html

https://pypi.python.org/pypi/FixedWidth/0.99.3


In [ ]:
lhr_lines = []
with open("heathrowdata.txt") as LHR_input:
    for row in LHR_input:
        print(row.strip("\n"))
        lhr_lines.append(row)
    

### Plain Python
* `split()` - funciona si el separador es espacio(s) en blanco y no hay espacios en los campos
* list of start/end positions or field lengths
* lists of lists vs. lists of dicts (o dictionary of dictionaries or of lists)

In [ ]:
with open("heathrowdata.txt") as LHR_input:
    # read the entire file
    LHR = LHR_input.read()
    
# explanation of the file    
lines= LHR.split("\n")[5:] 
header_1 = lines[0].split()
#print(header_1)
header_2 = [""] * 2 + lines[1].split()
#print(header_2) 
header = ["\n".join(x) for x in zip(header_1, header_2)]
#print(header)
lhr_list = [line.split() for line in lines[2:]]
lhr_tuple = [tuple(line.split()) for line in lines[2:]]



for row in lhr_tuple:
#for row in lhr_list:
    print(row)
    

In [ ]:

lhr_dict = [dict(zip(header_1, row))  for row in lhr_list]
for record in lhr_dict:
    print(record)


In [ ]:
field_lengths = (7, 4, 8, 8, 8, 8, 9)

start_end_pos = ((0, 7), (7, 11), (11, 19), (19, 27), (27, 35), (35, 43), (43, 52))

def normalize(field):
    try:
        return int(field.strip(" #" ))
    except ValueError as e:
        try:
            return float(field.strip())
        except ValueError as e:
            return None
    
with open("heathrowdata.txt") as LHR_input:
    LHR = LHR_input.read()
    lines = LHR.split("\n")[5:]
    records = []
    for line in lines[2:]:
        # using field lengths
        record = []
        pos = 0
        for field in field_lengths:
            record.append(line[pos : pos + field])
            pos += field
        # using start and end positions
        record = [normalize(line[a:b]) for a, b in start_end_pos]
        records.append(record)
        print(record)
  

In [ ]:
# list comprehension in a list comprehension
# shorter is not always better
lhr_records = [[normalize(line[a:b]) for a, b in start_end_pos] for line in lines[2:] if line.strip()]
lhr_records

In [ ]:
import pandas
infile_obj = open("heathrowdata.txt")
for i in range(5):
    next(infile_obj)


df_lhr = pandas.read_fwf(infile_obj)
print(df_lhr)


In [ ]:
! pip install fixedwidth

In [ ]:
from fixedwidth.fixedwidth import FixedWidth
import fixedwidth.fixedwidth as fixed

CONFIG = {
    "yyyy": {
        "required": True,
        "type": "integer",
        "start_pos": 1,
        "end_pos": 7,
        "alignment": "left",
        "padding": " ",
    },
    "mm": {
        "required": True,
        "type": "integer",
        "start_pos": 8,
        "end_pos": 11,
        "alignment": "left",
        "padding": " ",
    },
    "tmax": {
        "required": True,
        "type": "decimal",
        "start_pos": 12,
        "end_pos": 18,
        "alignment": "left",
        "padding": " ",
    },
    "tmin": {
        "required": True,
        "type": "decimal",
        "start_pos": 19,
        "end_pos": 26,
        "alignment": "left",
        "padding": " ",
    },
    "af": {
        "required": True,
        "type": "string",
        "start_pos": 27,
        "end_pos": 34,
        "alignment": "left",
        "padding": " ",
    },
    "rain": {
        "required": True,
        "type": "decimal",
        "start_pos": 35,
        "end_pos": 42,
        "alignment": "left",
        "padding": " ",
    },
    "sun": {
        "required": True,
        "type": "string",
        "start_pos": 43,
        "end_pos": 51,
        "alignment": "left",
        "padding": " ",
    },
}

In [ ]:
fw = FixedWidth(CONFIG)
with open("heathrowdata.txt") as LHR_input:
    LHR = LHR_input.read()
    lines= LHR.split("\n")[7:]
    for line in lines:
        if not line.strip():
            continue
        data = fw._string_to_dict(line)
        fw.update( **data)
        print(fw.data)

In [ ]:
fw.data 

In [ ]:
fw.line

In [ ]:
header_1

## Archivos delimitados (CSV)

* Un de los formats más comunes
* delimitado por un coma, tab, barra o otro carácter
* si un campo contiene el delimitador, hay que conlocar el campo entre comillas
* existe un módulo en la biblioteca estándar - https://docs.python.org/3/library/csv.html


In [ ]:
import requests
resultado = requests.get("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/05-24-2020.csv")

open("covid-19_2020-05-24.csv", "w").write(resultado.text)

In [ ]:
lines = []
with open("csv/temp_data_01.csv") as input_file:
    for row in input_file:
        lines.append(row)
        print(row.strip("\n")) 

### No re-implemente la biblioteca estándar
* ya se ha hecho
* se ha depurado y optimizado por los desarolladores core y los usuarios 


In [ ]:
# no haz esto jamás!

with open("csv/temp_data_01.csv") as input_file:
    records = [[field.strip('"\n') for field in row.split(",")] for row in input_file]
        
for record in records:
    print(record)


In [ ]:
def strip_commas(func):
    def wrapper(*params):
        result = func(*params)
        return result.replace(",", "")
    return wrapper

@strip_commas
def normalize(field):
    return field.replace("Missing", "")

import csv

with open("csv/temp_data_01.csv") as input_file:
    records = [[normalize(x) for x in row] for row in csv.reader(input_file)]
    
for record in records:
    print(record)


### comprensiones de listas  vs. expressiones generadores
* comprensiones de listas son la manera Pythonica para filtrar y hacer operaciones map
* expresiones generadores son equivalentes
  * comprensiones de listas requieren más memoria y son más rápidas
  * en el ejemplo encima, con un archivo muy grande la expresión externa podría requirer demasiada memoria

In [ ]:
# memory efficient version

def strip_commas(func):
    def wrapper(*params):
        result = func(*params)
        return result.replace(",", "")
    return wrapper

@strip_commas
def normalize(field):
    return field.replace("Missing", "")

import csv

with open("csv/temp_data_01.csv") as input_file:
    # creates a generator object but does NOT read from file here
    records_list = ([normalize(x) for x in row] for row 
                      in csv.reader(input_file))
    
    # print must now be in with block
    for record in records_list:
        print(record)


### Otros iteradores similares
* sort (cambia el objeto) vs sorted (iterador)
* reverse (cambia el objeto) vs reversed (iterador)
* itertools en la biblioteca estándar


### Otros delimitadores
* tab - `\t`
* barra - `|`

In [ ]:
# delimitado por tabs, encargado como texto plano

with open("csv/temp_data_01.txt") as input_file:
    for row in input_file:
        print(row.strip("\n"))

In [ ]:
# delimitado por tabs, encargado con módulo csv con tab como delimitador

import csv

with open("csv/temp_data_01.txt") as input_file:
    records = [row for row in csv.reader(input_file, delimiter="\t")]
    
for record in records:
    print(record)


In [ ]:
# delimitado por barras, encargado como texto plano

with open("csv/temp_data_pipes_01.txt") as input_file:
    for row in input_file:
        print(row.strip("\n"))

In [ ]:
# delimitado por barras, encargado con módulo csv y barra como delimitador

import csv

with open("csv/temp_data_pipes_01.txt") as input_file:
    records = [row for row in csv.reader(input_file, delimiter="|")]
    
for record in records:
    print(record)


### Common data structures
* lists of lists
* lists of tuples
* lists of dictionaries
* named tuples
* dataclasses (3.7 on)
* pandas dataframes

### Pandas y archivos CSV 

* puede encargarlo en un dataframe
* Usa el módulo csv module 
* Otros delimitadores se pueden usar, pero hay que espécificarlos

In [ ]:
import pandas
df = pandas.read_csv("csv/temp_data_01.csv")
#df = pandas.read_csv("csv/temp_data_pipes_01.txt", delimiter="|")
df

## Archivos de JSON

* Structured
* Allows nesting objects/types
* Flexible - elements of a "record" can change

### JSON vs SimpleJSON
* simplejson se añadió a la biblioteca estándar (llamado json) en Python 2.6 (2008)
* simplejson actualmente se actualiza más frecuente y tiene más functiones, e.g. decimal types

In [ ]:
with open("json/london.json") as input_file:
    for row in input_file:
        print(row.strip("\n"))

In [ ]:
# si preferias simplejson (si posible), but are okay failing back to json
try:
    import simplejson as json
except ImportError:
    import json

with open("json/london.json") as input_file:
    data = json.load(input_file)
    
data

In [ ]:

with open("owid-covid-data.json") as input_file:
    for number, row in enumerate(input_file):
        if number> 100:
            break
        print(row.strip("\n"))

In [ ]:
# si preferias simplejson (si posible), but are okay failing back to json
try:
    import simplejson as json
except ImportError:
    import json

with open("owid-covid-data.json") as input_file:
    data = json.load(input_file)
    
data["ABW"]

### Common json functions
* `json.loads(data_string)` - parses JSON string (or bytes or bytearrray) into dictionary
* `json.load(file)` - reads JSON string from (text or binary) file and parses into dictionary
* `json.dumps(a_dictionary)` - converts dictionary to serialized JSON **string**
* `json.dump(a_dictionary, file)` - parses JSON **string** into dictionary


### JSON file layouts
* one giant object
* array of objects
   * top level arrays subject to exploit, but still common
* one object per row - not really JSON, but common

In [ ]:
# top level array loaded as plain text

with open("json/chicago.json") as input_file:
    for row in input_file:
        print(row.strip("\n"))

In [ ]:
# top level array loaded as JSON object

with open("json/chicago.json") as input_file:
    data = json.load(input_file)
data

In [ ]:

with open("json/mars.json") as input_file:
    for row in input_file:
        print(row)

In [ ]:
with open("json/mars.json") as input_file:
    data = json.load(input_file)
data

## XML files

* xmltodict (via pip) - converts to dictionary
* elementree (standard library, need to walk tree/search for data


In [ ]:
with open("xml/sample_01.xml") as input_file:
    for row in input_file:
        print(row.strip("\n"))

In [ ]:
import xmltodict
with open("xml/sample_01.xml", "rb") as input_file:
    data = xmltodict.parse(input_file)
data

In [ ]:
import xml.etree.ElementTree as ET
tree = ET.parse('xml/sample_01.xml')
root = tree.getroot()
root

In [ ]:
for item in root:
    print(item.tag, item.attrib)

In [ ]:
with open("xml/observations.xml") as input_file:
    for row in input_file:
        print(row.strip("\n"))

## Optimizations

* avoid premature optimizations - start naively and then optimize
* processor time is generally cheaper than human time
* beware of loops - move things out
* avoid repeated string operations (concatenation, etc)
* parallelization and concurrency
* avoid reading and especially avoid writing to disk (or DB, or virtual memory)
* divide and conquer (map/reduce)
* sorting can help (binary search)
* generators save memory (but are a little slower)




## Tips and tricks

* Use commandline (unix) tools - e.g. sort, grep
* Clean lines rather than fields if possible
* Use sets 
    * for uniqueness
    * for membership
* Use dictionaries for matching
* Use list and dictionary comprehensions for speed
* Use generator expressions to save memory



In [ ]:
# sets vs. lists

numbers = list(range(100000))

def find_list(target, numbers):
    return (
        target in numbers
        and target - 1 in numbers
        and target - 2 in numbers
        and target - 3 in numbers
    )

def find_set(target, numbers):
    # convert numbers to a set! 
    numbers = set(numbers)
    return (
        target in numbers
        and target - 1 in numbers
        and target - 2 in numbers
        and target - 3 in numbers
    )

In [ ]:
%timeit find_list(99999, numbers)

%timeit find_set(99999, numbers)

In [ ]:
def join_generator(number):
    "-".join(str(n) for n in range(number))

def join_comprehension(number):
    "-".join([str(n) for n in range(number)])
    
def join_map(number):
    "-".join(map(str, range(100)))
    
def concat_str(n):
    result = ""
    for x in range(n-1):
        result += str(x) + "-"
    result += str(n-1)
    return result


%timeit join_generator(100)
%timeit join_comprehension(100)
%timeit join_map(100)
%timeit concat_str(100)

## Very large files

(Example: based on  product feed -> MongoDB, combining 4 files)
* 4 delimited flat files, unsorted
  * items - ~2 million rows; sku, description, categories, dimensions, compliance, brand, MPN, etc.
  * attributes - 20 million rows; sku, attr_id, attr_name, attr_value
  * alternates/accessories - sku, type, alt_sku - 1 million rows
  * cross reference - sku, competitor, alt part number - 
* Add record of any changes

### Solution
1. Sort all files by SKU using Unix sort
2. Get existing record from Mongo
3. Process all related files for that SKU
4. Compare for changes
5. Update Mongo if needed

### Debugging

#### **Use `print` a lot**
  * simple
  * fast
  * “Did the code get here, and what was x when it did?”
  * Not so good for edge cases in loops, large structures, etc.
  * a bit awkward to clean up afterwards

#### **logging** 
  * https://docs.python.org/3/howto/logging-cookbook.html, https://docs.python.org/3/howto/logging.html
  * useful in production
  * configurable levels
  * multiple handlers
  * file record
  * more work to set up
  * less clean-up, just set the debug levels


In [ ]:
import logging

# create the logger
logger = logging.getLogger("my_process")
logger.setLevel(logging.DEBUG)

# set up file for debug level messages
file_handler = logging.FileHandler("process.log")
file_handler.setLevel(logging.DEBUG)
logger.addHandler(file_handler)

# setup console for errors only
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.ERROR)
logger.addHandler(console_handler)

logger.debug("This goes only to the file")
logger.error("This only goes to the console and the file")

In [ ]:
print(open('process.log').read())

#### IDE debugger
  * conditional break points
  * stepping through a process
  * more work and time than print
  * IDE debuggers are a bit awkward for large files, etc

#### Python debugger
  * https://docs.python.org/3/library/pdb.html
  * good for complex situations
  * conditional break points
  * stepping through a process
  * more work and time than print
  * needs some clean-up

In [ ]:
import pdb

for x in range(10):
    # do various things here... 
    y = x*x
    breakpoint() 


## Testing?
* Problems with testing over very large data sets
* Small sample for sanity check
* Loud errors
* Be sparing with exceptions

